In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re


In [ ]:
# source_file = "https://escoe-website.s3.amazonaws.com/wp-content/uploads/2018/10/08164544/BGSCSV.zip"
# r = requests.get(source_file)
# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.extractall("downloads/")
# os.listdir("downloads/")



In [ ]:
from bgs.load_bgs_amounts import load_bgs_amounts

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

In [ ]:
tables.keys()

In [ ]:
conv = tables['Conventionals']

In [ ]:
conv.shape

In [ ]:
conv.head()

In [ ]:
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)


In [ ]:
conv.head()

In [ ]:
conv.sum(axis=1).head()

In [ ]:
sum_conv = tables["Sum of Conventionals"]
sum_conv.head()

In [ ]:
from bgs.load_bgs_prices import load_prices
price_df=load_prices("downloads/BGSPrices.csv")

In [ ]:
price_df.head()

In [ ]:
price_df.tail(20)

In [ ]:
price_df.index[0]

In [ ]:
c = pd.Series(pd.to_datetime(price_df.index, format="%d %b %Y")).dt.strftime("%b %Y").values
c[:10]

In [ ]:
c[0]

In [ ]:
conv[conv.index==c[0]]

In [ ]:
price_df[32300]

In [ ]:
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')

In [ ]:
price_df.columns = price_df.columns.map(str)
price_df['3500']

In [ ]:
price_strings = np.where(price_df.map(lambda x: type(x)==str))
price_strings

In [ ]:
c = Counter()
for i, x in enumerate(price_strings[0]):
    c[price_df.loc[price_df.index[x],price_df.columns[price_strings[1][i]]]]+=1
c.most_common(10)

In [ ]:
c.keys()

In [ ]:
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, None)
price_df = price_df.fillna(0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)

In [ ]:
conv.columns

In [ ]:
not_in_conventionals = set(price_df.columns.astype(str)) - set(conv.columns)

In [ ]:
from bgs.load_gilt_details import load_csv_blocks
details = load_csv_blocks("downloads/BGSDetails.csv")

In [ ]:
details['Conventionals'].head()

In [ ]:
details['Index-Linked Old-style'].head()

In [ ]:
details['Index-Linked New-style'].head()

In [ ]:
details['Conventionals']['Earliest redemption date']

In [ ]:
details['Conventionals']['Latest redemption date']

In [ ]:
details['Index-Linked Old-style'].index

In [ ]:
(set(details['Index-Linked Old-style'].index) | set(details['Index-Linked New-style'].index)) - not_in_conventionals

In [ ]:
def clean_date(bgs_index):
    if re.match(r'^\d{2}\s\w{3}\s+\d{4}$', bgs_index):
        return pd.to_datetime(bgs_index, format="%d %b %Y", errors='coerce').to_period('M').to_timestamp('M')
    else:
        return pd.to_datetime(bgs_index).to_period('M').to_timestamp('M')

conv.index = pd.Index([clean_date(x) for x in list(conv.index)])

In [ ]:
re.match(r'^\d{2}\s\w{3}\s+\d{4}$','30 Apr 2024')

In [ ]:
mtm_list = []

for bgs_gilt_id in conv.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([conv[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())
#mtm_df[bgs_gilt_id] = calc_df['amounts']*calc_df['prices']
#mtm_list.append(mtm_df)
conv_mtm_df = pd.concat(mtm_list, axis=1)


In [ ]:
conv_mtm_df.index

In [ ]:
conv_mtm_df.sum(axis=1)

In [ ]:
conv.index

In [ ]:
conv.loc['Dec 2024']

In [ ]:
conventionals = pd.DataFrame()
conventionals['Nominal'] = conv.sum(axis=1)
conventionals['MTM'] = conv_mtm_df.sum(axis=1)
conventionals.index = pd.to_datetime(conventionals.index, format="%b %Y").to_period('M').to_timestamp('M')
conventionals.plot()
# fig = plt.figure(figsize=(12, 6))
# ax = fig.add_subplot(111)
# ax.set_title("BGS MTM")
# ax.set_xlabel("Date")
# ax.set_ylabel("MTM")
# conv.sum(axis=1).plot(ax=ax, label='Conv')
# mtm_df.sum(axis=1).plot(ax=ax, label='MTM')
# ax.legend()

In [ ]:
(conventionals['Nominal'] - conventionals['MTM']).plot()

In [ ]:
conventionals['2021-09-01':].plot()

In [ ]:
old = tables['Calculated indexed nominal Old-style']

old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)

In [ ]:
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')

In [ ]:
for bgs_gilt_id in old.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([old[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())

In [ ]:
new = tables['Calculated indexed nominal New-style']
new.head()

In [ ]:
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)

new.index = pd.Index([clean_date(x) for x in list(new.index)])

for bgs_gilt_id in new.columns:
    # print(bgs_gilt_id)
    calc_df = pd.DataFrame(index=price_df.index, columns=['amounts','prices'])
    calc_df = pd.concat([new[bgs_gilt_id],price_df[bgs_gilt_id]], join='inner', axis=1, keys=['amounts','prices'])
    calc_df['mtm'] = calc_df['amounts']*calc_df['prices']/100
    mtm_list.append(calc_df['mtm'].rename(bgs_gilt_id).copy())

In [ ]:
new.head()

In [ ]:
total_gilts_mtm = pd.concat(mtm_list, axis=1)

total_gilts = pd.DataFrame()
total_gilts['MTM'] = total_gilts_mtm.sum(axis=1)
total_gilts.index = pd.to_datetime(total_gilts.index, format="%b %Y").to_period('M').to_timestamp('M')


In [ ]:
total_nominal = tables['Sum of total conventional and indexed-linked'].reset_index().set_index('index')
total_nominal = total_nominal.drop_duplicates()
total_nominal.index = pd.Index([clean_date(x) for x in list(total_nominal.index)])

total_nominal['Sum of total conventional and indexed-linked'] = pd.to_numeric(total_nominal['Sum of total conventional and indexed-linked'])
ax = total_nominal['Sum of total conventional and indexed-linked'].plot()
total_gilts.plot(ax = ax)

In [ ]:
total_nominal.head()

In [ ]:
total_nominal.drop_duplicates().index

In [ ]:
total_nominal.head()

In [ ]:
total_nominal['Sum of total conventional and indexed-linked']

In [ ]:
total_gilts

In [ ]:
total_nominal['Sum of total conventional and indexed-linked'] - total_gilts['MTM']

In [ ]:
p_apr25 = price_df.loc['2025-04-30']
p_apr25

In [ ]:
conv.loc['2025-04-30'].tail(20)

In [ ]:
new.index

In [ ]:
old.index

In [ ]:
details['Index-Linked New-style'].index

In [ ]:
disc_apr25 =  p_apr25.replace(0,None).map(lambda x: 100 - x if x and x < 100 else 0).fillna(0).T
disc_apr25

In [ ]:
conv_discount = (conv.loc['2025-04-30'].T * disc_apr25)/100
conv_discount = conv_discount[conv_discount>0].reset_index().rename(columns={'index':'Sequence',pd.to_datetime('2025-04-30'):'Discount'})
conv_discount


In [ ]:
conv_details = details['Conventionals']


In [ ]:
conv_details.loc[conv_details['Sequence']=='32295'].T

In [ ]:
disc_apr25['32295']

In [ ]:
conv.loc['Apr 2025']['32295']

In [ ]:
conv_dis = conv_discount.merge(conv_details[['Sequence','%','Latest redemption date', 'Issue date']], left_on='Sequence', right_on='Sequence')

In [ ]:
conv_dis['%'].value_counts()

In [ ]:
def clean_percentage(x):
    try:
        x = float(x)
    except (ValueError, TypeError):
        units, fractions = x.split(' ')
        num, denom = map(float, fractions.split('/'))
        x = float(units) + num / denom
    return x

conv_dis['%'] = conv_dis['%'].apply(clean_percentage).astype(float)

In [ ]:
conv_dis.groupby('%')['Discount'].sum().plot(kind='bar')

In [ ]:
conv_dis.groupby('%')['Discount'].sum().cumsum().plot(kind='bar')

In [ ]:
conv_dis.groupby('%')['Discount'].sum().sum()

In [ ]:
new_discount = (new.loc['30 Apr 2025'] * disc_apr25)/100
new_discount = new_discount[new_discount>0].reset_index().rename(columns={'index':'Sequence',pd.to_datetime('2025-04-30'):'Discount'})
new_discount

In [ ]:
new_details = details['Index-Linked New-style']
new_dis = new_discount.merge(new_details[['Sequence','%','Latest redemption date', 'Issue date']], left_on='Sequence', right_on='Sequence')
new_dis['%'] = new_dis['%'].apply(clean_percentage).astype(float)
new_dis.groupby('%')['Discount'].sum().cumsum().plot(kind='bar')

In [ ]:
old.tail()

In [ ]:
old_discount = (old.loc['Apr 2025'] * disc_apr25)/100
old_discount = old_discount[old_discount>0].reset_index().rename(columns={'index':'Sequence',0:'Discount'})
old_discount

In [ ]:
new_dis.groupby('%')['Discount'].sum().sum() + conv_dis.groupby('%')['Discount'].sum().sum()

In [ ]:
conv_dis['Issue date'] = conv_dis['Issue date'].apply(clean_date).dt.year

In [ ]:
conv_dis.sort_values('Issue date').groupby('Issue date')['Discount'].sum().plot(kind='bar')

In [ ]:
conv_dis['Latest redemption date'].to_list()

In [ ]:
conv_dis['Maturity date'] = conv_dis['Latest redemption date'].apply(clean_date).dt.year//5 * 5
conv_dis.sort_values('Maturity date').groupby('Maturity date')['Discount'].sum().plot(kind='bar')


In [ ]:
new_dis['Issue date'] = new_dis['Issue date'].apply(clean_date).dt.year
new_dis.sort_values('Issue date').groupby('Issue date')['Discount'].sum().plot(kind='bar')

In [ ]:
new_dis['Maturity date'] = new_dis['Latest redemption date'].apply(clean_date).dt.year//5 * 5
new_dis.sort_values('Maturity date').groupby('Maturity date')['Discount'].sum().plot(kind='bar')